In [ ]:
import os
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import requests


from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, cross_val_predict, train_test_split
from sklearn.metrics import (roc_auc_score, confusion_matrix, precision_score,
                             recall_score, roc_curve, precision_recall_curve,auc)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import fbeta_score

# from utils import plot_confusion_matrix

In [ ]:
dir_path = '/Users/alisa95/Downloads/FEATURES-2014-2015/'
names_list = sorted(os.listdir(dir_path)[::4])
sequences = pd.concat([pd.read_csv(dir_path + name, parse_dates=['timestamp'])for name in names_list], ignore_index=True)

window_size = 4
# размер окна
window_method = 1
# 0 - если хоть один элемент аномальный то окно аномально, 1 - если все, 2 если больше половины
experiment_message = 'тестируем лес'

In [ ]:
# сортировка
sequences.sort_values(['host', 'process', 'timestamp'], ascending=True, inplace=True)

In [ ]:
labels = sequences['isAnomaly']
sequences = sequences.drop(columns=['isAnomaly', 'host', 'process', 'timestamp'])

In [ ]:
window_sequences = pd.DataFrame()
window_labels = pd.DataFrame()
for i in range(window_size):
    window_sequences = pd.concat([window_sequences, sequences.shift(periods=i)], axis=1)
    window_labels[f'shifted_{i}'] = labels.shift(periods=i)

sequences = window_sequences
labels = window_labels

sequences.dropna(inplace=True)
labels.dropna(inplace=True)

num_columns = labels.shape[1]
sum_lables = labels.sum(axis=1)
if(window_method==0):
    for val_index in sum_lables.index:
        if sum_lables[val_index] != 0:
            sum_lables[val_index] = 1
elif(window_method==1):
    for val_index in sum_lables.index:
        if sum_lables[val_index] == num_columns:
            sum_lables[val_index] = 1
        else:
            sum_lables[val_index] = 0
            
labels=sum_lables

In [ ]:
%%time
forest = RandomForestClassifier(n_estimators=60, n_jobs=-1, random_state=1488)
forest_labels = cross_val_predict(forest, sequences, labels, cv=4,
                                  method='predict_proba')[:, 1]

In [ ]:
# precision recall
print("precision_score: ",precision_score(labels, forest_labels > 0.02), "\nrecall_score: ",recall_score(labels, forest_labels > 0.02))
# roc_auc_score
print("roc_auc_score: ",roc_auc_score(labels, forest_labels))
# roc_curve
fpr, tpr, _ = roc_curve(labels, forest_labels)
plt.plot(fpr, tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')

In [ ]:
# prc_curve
pr, rc, _ = precision_recall_curve(labels, forest_labels)
plt.plot(rc, pr)
plt.xlabel('rc')
plt.ylabel('pr')
print("prc_auc_score: ",auc(rc, pr))

In [ ]:
f2_score = fbeta_score(labels.astype(int), forest_labels.astype(int), beta=2)

In [ ]:
# RandomForestNotifyer
message='RandomForest done \n  window_size '+str(window_size)+ ' \n  window_method '+ str(window_method)+'\n  auc_roc '+str(roc_auc_score(labels, forest_labels)) +'\n  auc_rpc '+str(auc(rc, pr))+'\n  f2_score '+str(f2_score)+'\n'+experiment_message

r = requests.get('http://alarmerbot.ru/?key=c51ae3-7b05b8-ee307f&message='+str(message))

In [ ]:
for i in np.arange(0.00, 0.2, 0.0001):
    print('threshold:', i)
    x = confusion_matrix(labels, forest_labels < i).astype(float)
    print(x)
    x /= x.sum(axis=1)[:, None]
    print(x)
    print("pr: ",precision_score(labels, forest_labels < i).astype(float)," ; rc: " ,recall_score(labels, forest_labels < i).astype(float))
    print('\n')

In [ ]:
%%time
scaler = StandardScaler(copy=False)
logreg = LogisticRegression(solver='lbfgs', n_jobs=1)
logreg_pipe = Pipeline([('scaler', scaler), ('logreg', logreg)])
logreg_labels = cross_val_predict(logreg_pipe, sequences, labels,
                                  cv=4, method='predict_proba')[:, 1]

In [ ]:
%%time
boosting = LGBMClassifier(max_depth=7, n_estimators=80, n_jobs=-1)
boosting_labels = cross_val_predict(boosting, sequences, labels, cv=4,
                                    method='predict_proba')[:, 1]

In [ ]:
confusion_matrix(labels, forest_labels)

In [ ]:
precision_score(labels, forest_labels), recall_score(labels, forest_labels)

In [ ]:
confusion_matrix(labels, boosting_labels)

In [ ]:
precision_score(labels, boosting_labels), recall_score(labels, boosting_labels)

In [ ]:
confusion_matrix(labels, logreg_labels)

In [ ]:
precision_score(labels, logreg_labels), recall_score(labels, logreg_labels)

In [ ]:
class_names = np.array(["Healthy", "Anomaly"])
np.set_printoptions(precision=2)
plot_confusion_matrix(labels.astype(int), forest_labels.astype(int),
    classes=class_names, normalize=True, title="Forest Confusion Matrix")
plt.tight_layout()
plt.savefig("images/forest_confusion_matrix.png", dpi=140)

In [ ]:
class_names = np.array(["Healthy", "Anomaly"])
np.set_printoptions(precision=2)
plot_confusion_matrix(labels.astype(int), boosting_labels.astype(int),
    classes=class_names, normalize=True, title="Boosting Confusion Matrix")
plt.tight_layout()
plt.savefig("images/boosting_confusion_matrix.png", dpi=140)

In [ ]:
class_names = np.array(["Healthy", "Anomaly"])
np.set_printoptions(precision=2)
plot_confusion_matrix(labels.astype(int), logreg_labels.astype(int),
    classes=class_names, normalize=True, title="Logreg Confusion Matrix")
plt.tight_layout()
plt.savefig("images/logreg_confusion_matrix.png", dpi=140)

In [ ]:
sequences = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9],[10, 11, 12],[13, 14, 15]]),columns=['a', 'b', 'c'])
labels = pd.Series(np.array([0,0,1,1,0]))